## False Edges Expriment
Automated generation of a similarity matrix and graph for each recipe, adding increasing percentage of false edges, iterating over node based threshold, applying community detection algorithms and exporting results to csv file

In [1]:
from data.recipes import small_well_defined_recipe, small_weak_recipe, well_defined_recipe, weak_recipe, equal_recipe, random_recipe, big_well_defined_recipe, big_weak_recipe, big_equal_recipe, big_random_recipe
from classes.compound_generator import CompoundGenerator
from classes.graph import Graph
from classes.matrix import Matrix
import logging
from datetime import datetime
import os
import copy

In [2]:
base_dir = os.path.join("logs")
os.makedirs(base_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"/automated_graph_generation_{timestamp}.log"
logging.basicConfig(filename=base_dir+log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("+++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
recipes = [
    small_well_defined_recipe, 
    small_weak_recipe, 
    well_defined_recipe, 
    weak_recipe, 
    equal_recipe, 
    random_recipe, 
    big_well_defined_recipe, 
    big_weak_recipe, 
    big_equal_recipe, 
    big_random_recipe
]
noise = [True, False]
false_edges = [0, 10, 20, 30, 40, 50, 60, 70, 80]

def get_recipe_name(recipe):
    name=""
    for y in range(0, len(recipe), 1):
        name += str(recipe[y][1]['name']) + "-" + str(recipe[y][0]) + "-"
    name = name[:-1]
    return name

def log_run(recipe_name, false_edges):
    print(f"Log {recipe_name}, {false_edges}")
    logging.info(f"Recipe: {recipe_name}")
    logging.info("Normalized: True")
    logging.info(f"False Edges: {false_edges}%")
    logging.info("------------------------------------------------")
                    
def iterate_treshold(recipe_name, recipe, compounds_list, matrix):
    for percentage in false_edges:
        print(percentage)
        graph = Graph(recipe, compounds_list, matrix.similarity_matrix, matrix.options)
        graph.apply_matrix_treshold(80) 
        graph.apply_matrix_normalization()
        graph.create_graph()
        graph.add_false_edges(percentage)
        print("graph created")
        for x in range(0, 100, 5):
            print("X: ", x)
            graph1 = Graph(recipe, compounds_list, matrix.similarity_matrix, matrix.options, copy.deepcopy(graph.graph))
            graph1.options['graph']['false_edges'] = percentage
            graph1.apply_edge_treshold_node_based(x, 0)
            graph1.apply_edge_weight_normalization()
            graph1.apply_louvain()
            graph1.apply_lpa()
            graph1.apply_infomap()
            graph1.apply_greedy_modularity()
            graph1.calculate_scores()
            graph1.export_to_csv(folder_name=recipe_name)
        log_run(recipe_name, percentage)
       
for recipe in recipes:
    recipe_name = get_recipe_name(recipe)
    generator = CompoundGenerator(recipe, seed=0)
    compounds_list = generator.generate_compound_list()
    matrix = Matrix(compounds_list)
    print("matrix created")
   
    iterate_treshold(recipe_name, recipe, compounds_list, matrix)
            
